# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

awards_players_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/awards_players_cleaned.csv')
coaches_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/coaches_cleaned.csv')
players_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/players_cleaned.csv')
players_teams_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/players_teams_cleaned.csv')
series_post_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/series_post_cleaned.csv')
teams_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/teams_cleaned.csv')
teams_post_cleaned = pd.read_csv('../data/basketballPlayoffs_cleaned/teams_post_cleaned.csv')

# Obtain data from only 9 years

In [2]:
awards_players_model = awards_players_cleaned[awards_players_cleaned['year'] != 10]
coaches_model = coaches_cleaned[coaches_cleaned['year'] != 10]
players_teams_model = players_teams_cleaned[players_teams_cleaned['year'] != 10]
series_post_model = series_post_cleaned[series_post_cleaned['year'] != 10]
teams_model = teams_cleaned[teams_cleaned['year'] != 10]
teams_post_model = teams_post_cleaned[teams_post_cleaned['year'] != 10]

os.makedirs('../data/basketballPlayoffs_model', exist_ok=True)

awards_players_model.to_csv('../data/basketballPlayoffs_model/awards_players_model.csv', index=False)
coaches_model.to_csv('../data/basketballPlayoffs_model/coaches_model.csv', index=False)
players_cleaned.to_csv('../data/basketballPlayoffs_model/players_model.csv', index=False)
players_teams_model.to_csv('../data/basketballPlayoffs_model/players_teams_model.csv', index=False)
series_post_model.to_csv('../data/basketballPlayoffs_model/series_post_model.csv', index=False)
teams_model.to_csv('../data/basketballPlayoffs_model/teams_model.csv', index=False)
teams_post_model.to_csv('../data/basketballPlayoffs_model/teams_post_model.csv', index=False)

# Overalls from only 9 years

In [3]:
players_overall_avg = players_teams_model.groupby('playerID')['OVERALL'].mean().reset_index()

players_overall_avg.rename(columns={'OVERALL': 'OVERALL_ALL_TIME'}, inplace=True)

players_overall_avg['OVERALL_ALL_TIME'] = players_overall_avg['OVERALL_ALL_TIME'].round(1)

players_overall_avg.to_csv('../data/basketballPlayoffs_model/players_overall_all_time_model.csv', index=False)

In [4]:
rookie_players = players_teams_model[players_teams_model['is_rookie'] == 1]

rookie_overall_avg = rookie_players['OVERALL'].mean().round(1)

rookie_overall_avg_df = pd.DataFrame({'rookie_overall_avg': [rookie_overall_avg]})

rookie_overall_avg_df.to_csv('../data/basketballPlayoffs_model/rookie_overall_avg_model.csv', index=False)

In [5]:
rookie_coaches = coaches_model[coaches_model['is_rookie'] == 1]

rookie_overall_coaches_avg = rookie_coaches['OVERALL'].mean().round(1)

rookie_overall_coaches_avg_df = pd.DataFrame({'rookie_overall_coaches_avg': [rookie_overall_coaches_avg]})

rookie_overall_coaches_avg_df.to_csv('../data/basketballPlayoffs_model/rookie_overall_coaches_avg_model.csv', index=False)

In [6]:
coaches_overall_avg = coaches_model.groupby('coachID')['OVERALL'].mean().reset_index()

coaches_overall_avg.rename(columns={'OVERALL': 'OVERALL_ALL_TIME'}, inplace=True)

coaches_overall_avg['OVERALL_ALL_TIME'] = coaches_overall_avg['OVERALL_ALL_TIME'].round(1)

coaches_overall_avg.to_csv('../data/basketballPlayoffs_model/coaches_overall_all_time_model.csv', index=False)

# Obtain ranks from the 10th year to compare

In [7]:
playoffs_10th_year = teams_cleaned[teams_cleaned['year'] == 10][['tmID', 'playoff']]

playoffs_10th_year.to_csv('../data/basketballPlayoffs_model/playoffs_10th_year.csv', index=False)

# Obtain teams line up and coach from the 10th year

In [8]:
coaches_model_10th_year = coaches_cleaned[coaches_cleaned['year'] == 10][['coachID', 'tmID',]]

coaches_model_10th_year.to_csv('../data/basketballPlayoffs_model/coaches_model_10th_year.csv', index=False)

In [9]:
players_model_10th_year = players_teams_cleaned[players_teams_cleaned['year'] == 10][['playerID','tmID']]

players_model_10th_year = players_model_10th_year.sort_values(by='tmID')

players_model_10th_year.to_csv('../data/basketballPlayoffs_model/players_model_10th_year.csv', index=False)

# Add overalls calculated from 9 years to the 10th year players and coach

In [10]:
players_model_10th_year = pd.read_csv('../data/basketballPlayoffs_model/players_model_10th_year.csv')
players_overall_all_time_model = pd.read_csv('../data/basketballPlayoffs_model/players_overall_all_time_model.csv')

rookie_avg = 5.6 

players_model_10th_year = players_model_10th_year.merge(
    players_overall_all_time_model[['playerID', 'OVERALL_ALL_TIME']],
    on='playerID',
    how='left'
).fillna({'OVERALL_ALL_TIME': rookie_avg})

players_model_10th_year.rename(columns={'OVERALL_ALL_TIME': 'OVERALL'}, inplace=True)

players_model_10th_year.to_csv('../data/basketballPlayoffs_model/players_model_10th_year.csv', index=False)

In [11]:
coaches_model_10th_year = pd.read_csv('../data/basketballPlayoffs_model/coaches_model_10th_year.csv')
coaches_overall_all_time_model = pd.read_csv('../data/basketballPlayoffs_model/coaches_overall_all_time_model.csv')

rookie_avg_coach = 8.9 

coaches_model_10th_year = coaches_model_10th_year.merge(
    coaches_overall_all_time_model[['coachID', 'OVERALL_ALL_TIME']],
    on='coachID',
    how='left'
).fillna({'OVERALL_ALL_TIME': rookie_avg_coach})

coaches_model_10th_year.rename(columns={'OVERALL_ALL_TIME': 'OVERALL'}, inplace=True)

coaches_model_10th_year.to_csv('../data/basketballPlayoffs_model/coaches_model_10th_year.csv', index=False)


# Given the teams players composition and their respective overalls calculate the team overall (mean of all players)

In [12]:
players_model_10th_year = pd.read_csv('../data/basketballPlayoffs_model/players_model_10th_year.csv')

team_overall = players_model_10th_year.groupby('tmID')['OVERALL'].mean().reset_index()

team_overall['overall_team'] = team_overall['OVERALL'].round(1)

team_overall = team_overall.drop(columns=['OVERALL'])

team_overall.to_csv('../data/basketballPlayoffs_model/team_overall.csv', index=False)

# Given the team coaches and their respective overalls calculate the coach overall (mean of all coaches(1 or more than 1))

In [13]:
coaches_model_10th_year = pd.read_csv('../data/basketballPlayoffs_model/coaches_model_10th_year.csv')

team_overall_coaches = coaches_model_10th_year.groupby('tmID')['OVERALL'].mean().reset_index()

team_overall_coaches['overall_team_coach'] = team_overall_coaches['OVERALL'].round(1)

team_overall_coaches = team_overall_coaches.drop(columns=['OVERALL'])

team_overall_coaches.to_csv('../data/basketballPlayoffs_model/team_overall_coaches.csv', index=False)

# Create team final overall ( 90% team + 10% coach)

In [14]:
team_overall = pd.read_csv('../data/basketballPlayoffs_model/team_overall.csv')
team_overall_coach = pd.read_csv('../data/basketballPlayoffs_model/team_overall_coaches.csv')
teams_model = pd.read_csv('../data/basketballPlayoffs_model/teams_model.csv')

merged_df = pd.merge(team_overall, team_overall_coach, on='tmID')

merged_df['overall_team_final'] = (merged_df['overall_team'] * 0.9) + (merged_df['overall_team_coach'] * 0.1)
merged_df['overall_team_final'] = merged_df['overall_team_final'].round(2)

merged_df = pd.merge(merged_df, teams_model[['tmID', 'confID']], on='tmID', how='left')

final_df = merged_df[['tmID', 'confID', 'overall_team_final']]

final_df = final_df.drop_duplicates(subset='tmID')

final_df = final_df.sort_values(by='overall_team_final', ascending=True)

final_df.to_csv('../data/basketballPlayoffs_model/team_final_overall.csv', index=False)

# Accuracy

In [40]:
team_final_overall = pd.read_csv('../data/basketballPlayoffs_model/team_final_overall.csv')
playoffs_10th_year = pd.read_csv('../data/basketballPlayoffs_model/playoffs_10th_year.csv')

top_teams_by_conf = team_final_overall.sort_values(by='overall_team_final', ascending=False).groupby('confID').head(4)

print("Teams that go to the playoffs acording to the previsions:\n")
print(top_teams_by_conf)

comparison = top_teams_by_conf.merge(playoffs_10th_year, on='tmID', how='left')

correct_predictions = comparison[comparison['playoff'] == 1]

print("\nTeams that are in the playoffs and in the previsions:\n")
print(correct_predictions)

accuracy = len(correct_predictions) / len(top_teams_by_conf) * 100
print(f"\nThe accuracy is {accuracy:.2f}%")


Teams that go to the playoffs acording to the previsions:

    tmID  confID  overall_team_final
12     8       1               11.17
11    16       1                9.13
10     5       0                8.95
9     13       1                8.68
8     17       1                8.54
5     11       0                8.28
4      7       0                8.21
3     19       0                8.18

Teams that are in the playoffs and in the previsions:

   tmID  confID  overall_team_final  playoff
0     8       1               11.17        1
1    16       1                9.13        1
2     5       0                8.95        1
3    13       1                8.68        1
4    17       1                8.54        1
6     7       0                8.21        1
7    19       0                8.18        1

The accuracy is 87.50%


## SVM MODEL

In [113]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the data
team_final_overall = pd.read_csv('../data/basketballPlayoffs_model/team_final_overall.csv')
playoffs_10th_year = pd.read_csv('../data/basketballPlayoffs_model/playoffs_10th_year.csv')

# Merge the overall data with the playoff data
data = pd.merge(team_final_overall, playoffs_10th_year, on='tmID', how='inner')

# Split the data into predictor variables (overalls) and target variables (playoffs)
X = data[['overall_team_final']]  # The predictor variable (team's overall average)
y = data['playoff']  # The target variable (1 if the team made it to the playoffs, 0 otherwise)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the SVM model with probability enabled
svm_model = SVC(kernel='linear', probability=True)  # Enable probability=True
svm_model.fit(X_train, y_train)

# Make probability predictions for all teams (not just the test set)
probabilities = svm_model.predict_proba(X)[:, 1]  # Probability of the team making the playoffs

# Split the data into two conferences: confID == 1 and confID == 0
conf_1_data = data[data['confID'] == 1]
conf_0_data = data[data['confID'] == 0]

# Normalize the probabilities within each conference
conf_1_probabilities = (probabilities[conf_1_data.index] - probabilities[conf_1_data.index].min()) / (probabilities[conf_1_data.index].max() - probabilities[conf_1_data.index].min())  # Normalize between 0 and 1
conf_0_probabilities = (probabilities[conf_0_data.index] - probabilities[conf_0_data.index].min()) / (probabilities[conf_0_data.index].max() - probabilities[conf_0_data.index].min())  # Normalize between 0 and 1

# Adjust the probabilities so that their sum is equal to 8, separately for each conference
conf_1_probability_sum = conf_1_probabilities.sum()
conf_0_probability_sum = conf_0_probabilities.sum()

# Adjust the probabilities so that the sum equals 8 within each conference
adjusted_conf_1_probabilities = conf_1_probabilities * 8 / conf_1_probability_sum
adjusted_conf_0_probabilities = conf_0_probabilities * 8 / conf_0_probability_sum

# **Adjust probabilities based on the "overall_team_final"**
# Calculate the probabilities directly based on "overall_team_final" for each conference
conf_1_data = conf_1_data.copy()  # Create an explicit copy
conf_0_data = conf_0_data.copy()  # Create an explicit copy

conf_1_data['adjusted_probability'] = conf_1_data['overall_team_final'] / conf_1_data['overall_team_final'].max() / 2
conf_0_data['adjusted_probability'] = conf_0_data['overall_team_final'] / conf_0_data['overall_team_final'].max() / 2

conf_1_data['probability'] = adjusted_conf_1_probabilities
conf_0_data['probability'] = adjusted_conf_0_probabilities

conf_1_data['predicted'] = 0
conf_0_data['predicted'] = 0

# Add predictions to the dataset, considering the top 4 teams per conference
for conf_data in [conf_1_data, conf_0_data]:
    top_teams = conf_data.nlargest(4, 'overall_team_final')  # Select the top 4 teams
    conf_data.loc[top_teams.index, 'predicted'] = 1  # Mark the top 4 teams as 1 (make the playoffs)

# Concatenate the data from both conferences
final_data = pd.concat([conf_1_data, conf_0_data])

# Round the 'adjusted_probability' to 2 decimal places
final_data['adjusted_probability'] = final_data['adjusted_probability'].round(2)

# Sort the teams by adjusted_probability in descending order
final_data_sorted = final_data.sort_values(by='adjusted_probability', ascending=False)

# Display all teams and their adjusted probabilities, predictions, etc., sorted by adjusted_probability
print("Final Table with Teams and their Adjusted Probabilities (Sorted by Adjusted Probability):")
print(final_data_sorted[['tmID', 'confID', 'predicted', 'playoff', 'adjusted_probability', 'overall_team_final']])

# Compute the overall accuracy (for all teams combined, not per conference)
overall_accuracy = accuracy_score(final_data['playoff'], final_data['predicted'])

print(f"\nOverall Accuracy of prediction vs actual: {overall_accuracy:.2f}")


Final Table with Teams and their Adjusted Probabilities (Sorted by Adjusted Probability):
    tmID  confID  predicted  playoff  adjusted_probability  overall_team_final
12     8       1          1        1                  0.50               11.17
10     5       0          1        1                  0.50                8.95
3     19       0          1        1                  0.46                8.18
4      7       0          1        1                  0.46                8.21
5     11       0          1        0                  0.46                8.28
2      4       0          0        0                  0.45                8.06
1      0       0          0        1                  0.45                7.97
0      2       0          0        0                  0.44                7.90
11    16       1          1        1                  0.41                9.13
9     13       1          1        1                  0.39                8.68
7     10       1          0        0     

## KNN Model

In [112]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the data
team_final_overall = pd.read_csv('../data/basketballPlayoffs_model/team_final_overall.csv')
playoffs_10th_year = pd.read_csv('../data/basketballPlayoffs_model/playoffs_10th_year.csv')

# Merge the overall data with the playoff data
data = pd.merge(team_final_overall, playoffs_10th_year, on='tmID', how='inner')

# Split the data into predictor variables (overalls) and target variables (playoffs)
X = data[['overall_team_final']]  # The predictor variable (team's overall average)
y = data['playoff']  # The target variable (1 if the team made it to the playoffs, 0 otherwise)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the KNN model
knn_model = KNeighborsClassifier(n_neighbors=3)  # Using 3 nearest neighbors
knn_model.fit(X_train, y_train)

# Make predictions for all teams (not just the test set)
probabilities = knn_model.predict_proba(X)[:, 1]  # Probability of the team making the playoffs

# Split the data into two conferences: confID == 1 and confID == 0
conf_1_data = data[data['confID'] == 1]
conf_0_data = data[data['confID'] == 0]

# Normalize the probabilities within each conference
conf_1_probabilities = (probabilities[conf_1_data.index] - probabilities[conf_1_data.index].min()) / (probabilities[conf_1_data.index].max() - probabilities[conf_1_data.index].min())  # Normalize between 0 and 1
conf_0_probabilities = (probabilities[conf_0_data.index] - probabilities[conf_0_data.index].min()) / (probabilities[conf_0_data.index].max() - probabilities[conf_0_data.index].min())  # Normalize between 0 and 1

# Adjust the probabilities so that their sum is equal to 8, separately for each conference
conf_1_probability_sum = conf_1_probabilities.sum()
conf_0_probability_sum = conf_0_probabilities.sum()

# Adjust the probabilities so that the sum equals 8 within each conference
adjusted_conf_1_probabilities = conf_1_probabilities * 8 / conf_1_probability_sum
adjusted_conf_0_probabilities = conf_0_probabilities * 8 / conf_0_probability_sum

# **Adjust probabilities based on the "overall_team_final"**
# Calculate the probabilities directly based on "overall_team_final" for each conference
conf_1_data = conf_1_data.copy()  # Create an explicit copy
conf_0_data = conf_0_data.copy()  # Create an explicit copy

conf_1_data['adjusted_probability'] = conf_1_data['overall_team_final'] / conf_1_data['overall_team_final'].max() / 2
conf_0_data['adjusted_probability'] = conf_0_data['overall_team_final'] / conf_0_data['overall_team_final'].max() / 2

conf_1_data['probability'] = adjusted_conf_1_probabilities
conf_0_data['probability'] = adjusted_conf_0_probabilities

conf_1_data['predicted'] = 0
conf_0_data['predicted'] = 0

# Add predictions to the dataset, considering the top 4 teams per conference
for conf_data in [conf_1_data, conf_0_data]:
    top_teams = conf_data.nlargest(4, 'overall_team_final')  # Select the top 4 teams
    conf_data.loc[top_teams.index, 'predicted'] = 1  # Mark the top 4 teams as 1 (make the playoffs)

# Concatenate the data from both conferences
final_data = pd.concat([conf_1_data, conf_0_data])

# Round the 'adjusted_probability' to 2 decimal places
final_data['adjusted_probability'] = final_data['adjusted_probability'].round(2)

# Sort the teams by adjusted_probability in descending order
final_data_sorted = final_data.sort_values(by='adjusted_probability', ascending=False)

# Display all teams and their adjusted probabilities, predictions, etc., sorted by adjusted_probability
print("Final Table with Teams and their Adjusted Probabilities (Sorted by Probability):")
print(final_data_sorted[['tmID', 'confID', 'predicted', 'playoff', 'adjusted_probability', 'overall_team_final']])

# Compute the overall accuracy (for all teams combined, not per conference)
overall_accuracy = accuracy_score(final_data['playoff'], final_data['predicted'])

print(f"\nOverall Accuracy of prediction vs actual: {overall_accuracy:.2f}")


Final Table with Teams and their Adjusted Probabilities (Sorted by Probability):
    tmID  confID  predicted  playoff  adjusted_probability  overall_team_final
12     8       1          1        1                  0.50               11.17
10     5       0          1        1                  0.50                8.95
3     19       0          1        1                  0.46                8.18
4      7       0          1        1                  0.46                8.21
5     11       0          1        0                  0.46                8.28
2      4       0          0        0                  0.45                8.06
1      0       0          0        1                  0.45                7.97
0      2       0          0        0                  0.44                7.90
11    16       1          1        1                  0.41                9.13
9     13       1          1        1                  0.39                8.68
7     10       1          0        0              

## RFC (Random Forest Classifier)

In [115]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the data
team_final_overall = pd.read_csv('../data/basketballPlayoffs_model/team_final_overall.csv')
playoffs_10th_year = pd.read_csv('../data/basketballPlayoffs_model/playoffs_10th_year.csv')

# Merge the overall data with the playoff data
data = pd.merge(team_final_overall, playoffs_10th_year, on='tmID', how='inner')

# Split the data into predictor variables (overalls) and target variables (playoffs)
X = data[['overall_team_final']]  # The predictor variable (team's overall average)
y = data['playoff']  # The target variable (1 if the team made it to the playoffs, 0 otherwise)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # 100 trees in the forest
rf_model.fit(X_train, y_train)

# Use CalibratedClassifierCV to enable probability estimates
calibrated_rf_model = CalibratedClassifierCV(rf_model, method='sigmoid', cv='prefit')
calibrated_rf_model.fit(X_train, y_train)

# Make probability predictions for all teams (not just the test set)
probabilities = calibrated_rf_model.predict_proba(X)[:, 1]  # Probability of the team making the playoffs

# Split the data into two conferences: confID == 1 and confID == 0
conf_1_data = data[data['confID'] == 1]
conf_0_data = data[data['confID'] == 0]

# Normalize the probabilities within each conference
conf_1_probabilities = (probabilities[conf_1_data.index] - probabilities[conf_1_data.index].min()) / (probabilities[conf_1_data.index].max() - probabilities[conf_1_data.index].min())  # Normalize between 0 and 1
conf_0_probabilities = (probabilities[conf_0_data.index] - probabilities[conf_0_data.index].min()) / (probabilities[conf_0_data.index].max() - probabilities[conf_0_data.index].min())  # Normalize between 0 and 1

# Adjust the probabilities so that their sum is equal to 8, separately for each conference
conf_1_probability_sum = conf_1_probabilities.sum()
conf_0_probability_sum = conf_0_probabilities.sum()

# Adjust the probabilities so that the sum equals 8 within each conference
adjusted_conf_1_probabilities = conf_1_probabilities * 8 / conf_1_probability_sum
adjusted_conf_0_probabilities = conf_0_probabilities * 8 / conf_0_probability_sum

# **Adjust probabilities based on the "overall_team_final"**
# Calculate the probabilities directly based on "overall_team_final" for each conference
conf_1_data = conf_1_data.copy()  # Create an explicit copy
conf_0_data = conf_0_data.copy()  # Create an explicit copy

conf_1_data['adjusted_probability'] = conf_1_data['overall_team_final'] / conf_1_data['overall_team_final'].max() / 2
conf_0_data['adjusted_probability'] = conf_0_data['overall_team_final'] / conf_0_data['overall_team_final'].max() / 2

conf_1_data['probability'] = adjusted_conf_1_probabilities
conf_0_data['probability'] = adjusted_conf_0_probabilities

conf_1_data['predicted'] = 0
conf_0_data['predicted'] = 0

# Add predictions to the dataset, considering the top 4 teams per conference
for conf_data in [conf_1_data, conf_0_data]:
    top_teams = conf_data.nlargest(4, 'overall_team_final')  # Select the top 4 teams
    conf_data.loc[top_teams.index, 'predicted'] = 1  # Mark the top 4 teams as 1 (make the playoffs)

# Concatenate the data from both conferences
final_data = pd.concat([conf_1_data, conf_0_data])

# Round the 'adjusted_probability' to 2 decimal places
final_data['adjusted_probability'] = final_data['adjusted_probability'].round(2)

# Sort the teams by adjusted_probability in descending order
final_data_sorted = final_data.sort_values(by='adjusted_probability', ascending=False)

# Display all teams and their adjusted probabilities, predictions, etc., sorted by adjusted_probability
print("Final Table with Teams and their Adjusted Probabilities (Sorted by Probability):")
print(final_data_sorted[['tmID', 'confID', 'predicted', 'playoff', 'adjusted_probability', 'overall_team_final']])

# Compute the overall accuracy (for all teams combined, not per conference)
overall_accuracy = accuracy_score(final_data['playoff'], final_data['predicted'])

print(f"\nOverall Accuracy of prediction vs actual: {overall_accuracy:.2f}")


Final Table with Teams and their Adjusted Probabilities (Sorted by Probability):
    tmID  confID  predicted  playoff  adjusted_probability  overall_team_final
12     8       1          1        1                  0.50               11.17
10     5       0          1        1                  0.50                8.95
3     19       0          1        1                  0.46                8.18
4      7       0          1        1                  0.46                8.21
5     11       0          1        0                  0.46                8.28
2      4       0          0        0                  0.45                8.06
1      0       0          0        1                  0.45                7.97
0      2       0          0        0                  0.44                7.90
11    16       1          1        1                  0.41                9.13
9     13       1          1        1                  0.39                8.68
7     10       1          0        0              